In [1]:
!pip install fpdf


In [2]:
pip install kaleido==0.1.0post1

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datetime import date
from pathlib import Path
import sqlite3
import pandas as pd  
import plotly.express as px 
from fpdf import FPDF 
import kaleido

# Define paths and chart style

In [4]:
# Define the plotly template. Some other examples:
plotly_template = "presentation"

In [5]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

# Total Sales by Month

In [6]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [7]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [8]:
# Print the DataFrame
print(df)

      sale_date  total_sales
0    2022-01-01         1900
1    2022-01-02         1355
2    2022-01-03          650
3    2022-01-04          975
4    2022-01-05          330
..          ...          ...
335  2022-12-26          925
336  2022-12-27          575
337  2022-12-28          550
338  2022-12-29          795
339  2022-12-31         1255

[340 rows x 2 columns]


In [9]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    340 non-null    object
 1   total_sales  340 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [10]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    340 non-null    datetime64[ns]
 1   total_sales  340 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.4 KB


In [11]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2022-01-01,1900
2022-01-02,1355
2022-01-03,650


In [12]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2022-01-31,23175
2022-02-28,19990
2022-03-31,24065
2022-04-30,24615
2022-05-31,25180
2022-06-30,21205
2022-07-31,22330
2022-08-31,22005
2022-09-30,23735


In [13]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,23175,Jan
2022-02-28,19990,Feb
2022-03-31,24065,Mar
2022-04-30,24615,Apr
2022-05-31,25180,May
2022-06-30,21205,Jun
2022-07-31,22330,Jul
2022-08-31,22005,Aug
2022-09-30,23735,Sep


In [17]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
    
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

# Total Sales by Product

In [18]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)

In [19]:
df

,product_name,total_sales
0,Product A,59250
1,Product B,28300
2,Product C,89925
3,Product D,39560
4,Product E,69540


In [20]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

# Top Customer by Sales

In [21]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [22]:
df

,customer_name,total_sales
0,Jane Doe,37705
1,Frank Wilson,32990
2,Bob Smith,30175
3,David Brown,29830
4,Alice Jones,27980
5,Isabel Garcia,26975
6,John Doe,26720
7,Emily Davis,26085
8,Grace Lee,25605
9,Henry Chen,22510


In [23]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

In [24]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

''